In [1]:
!nvidia-smi

Mon Oct 12 14:19:07 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/My Drive/AnomalyDetectionCVPR2018-Pytorch

/content/drive/My Drive/AnomalyDetectionCVPR2018-Pytorch


In [4]:
!pip install av

     |████████████████████████████████| 36.9MB 80kB/s 


In [5]:
import argparse
import os
from os import path, mkdir
import numpy as np
import torch
import torch.backends.cudnn as cudnn
from tqdm import tqdm
from data_loader import VideoIter
from network.c3d import C3D
from utils.utils import set_logger, build_transforms

In [6]:
debug_mode = True 
dataset_path = 'Test-Videos/'  # give your path 
clip_length = 16
num_workers = 32
frame_interval = 1
log_file = ''
annotation_path = '/content/drive/My Drive/AnomalyDetectionCVPR2018-Pytorch/Test_Annotation.txt'
save_dir = 'test_features'

pretrained_3d = '/content/drive/My Drive/AnomalyDetectionCVPR2018-Pytorch/pretrained_c3d_model/c3d.pickle'
resume_epoch= -1
batch_size = 8 
random_seed = 1

current_path = None
current_dir = None
current_data = None

In [7]:
class FeaturesWriter:
	def __init__(self, chunk_size=16):
		self.path = None
		self.dir = None
		self.data = None
		self.chunk_size = chunk_size

	def _init_video(self, video_name, dir):
		self.path = path.join(dir, f"{video_name}.txt")
		self.dir = dir
		self.data = dict()

	def has_video(self):
		return self.data is not None

	def dump(self):
		print(f'Dumping {self.path}')
		if not path.exists(self.dir):
			os.mkdir(self.dir)

		features = np.array([self.data[key] for key in sorted(self.data)])
		features = features / np.expand_dims(np.linalg.norm(features, ord=2, axis=-1), axis=-1)
		padding_count = int(32 * np.ceil(features.shape[0] / 32) - features.shape[0])
		features = torch.from_numpy(np.vstack([features, torch.zeros(padding_count, 4096)]))
		segments = torch.stack(torch.chunk(features, chunks=32, dim=0))
		avg_segments = segments.mean(dim=-2).numpy()
		with open(self.path, 'w') as fp:
			for d in avg_segments:
				d = [str(x) for x in d]
				fp.write(' '.join(d) + '\n')

	def _is_new_video(self, video_name, dir):
		new_path = path.join(dir, f"{video_name}.txt")
		if self.path != new_path and self.path is not None:
			return True

		return False

	def store(self, feature, idx):
		self.data[idx] = list(feature)

	def write(self, feature, video_name, idx, dir):
		if not self.has_video():
			self._init_video(video_name, dir)

		if self._is_new_video(video_name, dir):
			self.dump()
			self._init_video(video_name, dir)

		self.store(feature, idx)

In [8]:

def read_features(video_name, dir):
	file_path = f"{video_name}.txt"
	file_path = path.join(dir, file_path)
	if not path.exists(file_path):
		raise Exception(f"Feature doesn't exist: {file_path}")
	features = None
	with open(file_path, 'r') as fp:
		data = fp.read().splitlines(keepends=False)
		features = np.zeros((len(data), 4096))
		for i, line in enumerate(data):
			features[i, :] = [float(x) for x in line.split(' ')]

	return torch.from_numpy(features)


In [10]:
def main():
	device = torch.device("cuda" if torch.cuda.is_available()
						else "cpu")

	set_logger(log_file=log_file, debug_mode=debug_mode)

	torch.manual_seed(random_seed)
	torch.cuda.manual_seed(random_seed)
	cudnn.benchmark = True

	train_loader = VideoIter(dataset_path=dataset_path,
							annotation_path=annotation_path,
							clip_length=clip_length,
							frame_stride=frame_interval,
							video_transform=build_transforms(),
							name='Features extraction')

	train_iter = torch.utils.data.DataLoader(train_loader,
											batch_size=batch_size,
											shuffle=False,
											num_workers=num_workers,  # 4, # change this part accordingly
											pin_memory=True)

	network = C3D(pretrained=pretrained_3d)
	network = network.to(device)

	if not path.exists(save_dir):
		mkdir(save_dir)

	features_writer = FeaturesWriter()

	for i_batch, (data, target, sampled_idx, dirs, vid_names) in tqdm(enumerate(train_iter)):
		with torch.no_grad():
			outputs = network(data.to(device)).detach().cpu().numpy()

			for i, (dir, vid_name, start_frame) in enumerate(zip(dirs, vid_names, sampled_idx.cpu().numpy())):
				dir = path.join(save_dir, dir)
				features_writer.write(feature=outputs[i], video_name=vid_name, idx=start_frame, dir=dir)

	features_writer.dump()

In [11]:
if __name__ == "__main__":
  main()

single loader used


2020-10-12 14:24:18: VideoIter:: iterator initialized (phase: 'Features extraction', num: 96)



no corresponding:: fc7.weight
no corresponding:: fc7.bias
no corresponding:: fc8.weight
no corresponding:: fc8.bias


0it [00:00, ?it/s]/usr/local/lib/python3.6/dist-packages/torchvision/io/video.py:106: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  + "follow-up version. Please use pts_unit 'sec'."
/usr/local/lib/python3.6/dist-packages/torchvision/io/video.py:106: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  + "follow-up version. Please use pts_unit 'sec'."
/usr/local/lib/python3.6/dist-packages/torchvision/io/video.py:106: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  + "follow-up version. Please use pts_unit 'sec'."
/usr/local/lib/python3.6/dist-packages/torchvision/io/video.py:106: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  + "follow-up version. Please use pts_unit 'sec'."
/usr/local

Dumping test_features/Test-Videos/Normal_Videos_576_x264.txt


...

Dumping test_features/Test-Videos/Normal_Videos_869_x264.txt


2150it [35:25,  2.02it/s]

Dumping test_features/Test-Videos/Normal_Videos_870_x264.txt


2158it [35:30,  1.01it/s]

Dumping test_features/Test-Videos/Normal_Videos_010_x264.txt
